# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **

This project is a classification problem since we are trying to determine a discrete label of yes or no whether students need early intervention.

## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"

Student data read successfully!


### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [2]:
# TODO: Calculate number of students
n_students = len(student_data)

# TODO: Calculate number of features
n_features = len(student_data.columns)-1

# TODO: Calculate passing students
n_passed = sum(student_data['passed'] == 'yes')

# TODO: Calculate failing students
n_failed = sum(student_data['passed'] == 'no')

# TODO: Calculate graduation rate
grad_rate = (100.0*n_passed)/(n_students )

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)

Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.

In [3]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()

Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [4]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [7]:
# TODO: Import any additional functionality you may need here
from sklearn.model_selection import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = num_test, random_state=42)

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

**Answer: **

Gaussian Naive Bayes:
    
Describe one real-world application in industry where the model can be applied. (You may need to do a small bit of research for this — give references!)
What are the strengths of the model; when does it perform well?
What are the weaknesses of the model; when does it perform poorly?
What makes this model a good candidate for the problem, given what you know about the data?

1. Naive Baye is commonly used in industry for text analysis such as for Google's gmail spam email detection. Each word in the text can be a probabilitic indicator whether the email was spam (for example the use of the word viagra migh give assign the email as spam probability but perhaps not the name of a friend or loved one). 

2. In practice the algorithm works relevatively well on classification tasks with datasets that have a large amount of features without having to understand or model any of the complex relationships that might exist. Since the model assumes the features are independent it can make the model easy to understand and efficient to training.

3. The algorithm is extremely naive thus it cannot capture important feature interactions that might exist or model hierarchical functioning.

4. The student intervention dataset has a lot of features with limited data. Thus GNB might be a good first start to see how well a naive model might predict the student graduation rates without dealing with the bias-variance dilemma. The F1 score can serve as a good benchmark standard of other model to match or exceed.

Decision Trees:

Describe one real-world application in industry where the model can be applied. (You may need to do a small bit of research for this — give references!)
What are the strengths of the model; when does it perform well?
What are the weaknesses of the model; when does it perform poorly?
What makes this model a good candidate for the problem, given what you know about the data?

1. Decision Trees are often used in expert knowledge systems to help diagnoise a problem (for example common illnesses in the medical field by asking a lot of highly relevant questions that reduce uncertainty of the end result).

2. The algorithm is easy to understand, requires little data preparation with its hierarchical list of if-else rules to break down a problem until it has enough information to determine the most likely result. When fully trained the model is extremely fast and efficient at making predictions.

3. Can generate very deep and complex trees, but will start to capture noise rather than the underyline relationships in the data if tree is too complex. Most decision-tree algorithms use a greed algorithm that make local optimal decisions will not necessary generate the global optimal. The algorithm also struggles to learn complex functions such as XOR or multiplexer problems or if the labels or severly unbalanced.

4. The algorithm generates very fast prediction by looking at the most essential features using entrophy. Therefore if we assume a one time fixed cost on training time it might be a good model to reduce computational resources when querying student predictions for the school while also serving as another benchmark for F1 score performance.

Ensemble Methods (AdaBoost):

Describe one real-world application in industry where the model can be applied. (You may need to do a small bit of research for this — give references!)
What are the strengths of the model; when does it perform well?
What are the weaknesses of the model; when does it perform poorly?
What makes this model a good candidate for the problem, given what you know about the data?

1. Some real-world applications of adaboosting are in biology, computer vision, and speech processing.

2. Adaboost is generally good at handling mix data types and robust to overfitting.

3. The algorithm takes up a lot of computational resources and is not very scalable since processes data sequentially.

4. Though Adaboost takes up a lot of resources it tends to find an optimal model without overfitting which might be attractice if we want to reach a 95% graduation rate without overfitting the small dataset provided.

K-Nearest Neighbors:

Describe one real-world application in industry where the model can be applied. (You may need to do a small bit of research for this — give references!)
What are the strengths of the model; when does it perform well?
What are the weaknesses of the model; when does it perform poorly?
What makes this model a good candidate for the problem, given what you know about the data?

1. K-Nearest Neighbors is used in applications where you are searching for similar items such as online document searches or for recommendations systems (if a person like some movies it possible they might like similar movies of the same genre, casting, director or storyplot).

2. Simple algorithm to implement and understand, remembers old data, fast to train

3. Takes up a large amount of memory for larger datasets and computationally expensive for making predictions not seen in the data. Sensitive to the local structure of the data so might not be able to capture more in depth relationships that might exist.

4. I decided to try knn since it tries to bundle similar data point together I would imagine that students demonstrate the same behaviors in school are likely to have similar gradutations rates.

Stochastic Gradient Descent (SGDC):
    
1. SGDC is used in all kinds machine learning models to iterative improve the scoring function. For example deep learning models uses stochastic gradient descent for large scale problems such as driverless cars where iteratively going through the entire dataset update each parameter would not be practically feastible.

2. SGDC is very efficient with lots of parameters available to tune.

3. Algorithm is sensitive to scaling and requires a number of hyperparameters.

4. While SGDC has a lot of parameters it is very efficient and flexible which might be useful if we need to tune the model to accurately model student intervention.


Support Vector Machine:
    
Describe one real-world application in industry where the model can be applied. (You may need to do a small bit of research for this — give references!)
What are the strengths of the model; when does it perform well?
What are the weaknesses of the model; when does it perform poorly?
What makes this model a good candidate for the problem, given what you know about the data?

1. Support Vector Machines are commonly used in industry for tasks such as image classification or bioinformatics to separate and identify various types of proteins or cancers.

2. With the additional use of the kernel trick, SVMs are effective in high dimensional spaces. Once the model is fully trained it only keeps support vector which makes it extremely memory efficient (when making predictions).

3. Support Vector Machine use quadratic programing which does not scale well in speed or memory (training) for large-scale tasks.

4. I decided to use an SVC since the algorithm is good at finding an optimal decision boundary with little memory usage once trained. I figure it might be able to provide highly accurate prediction of intervention that may prove more useful than the memory usage it ends up taking.

Logistic Regression:
    
1. Logistic Regression is a very simple linear classification model that is used often used in more complex models such as neural networks to learn more complex functions such as voice or hand recognition, but can also be used for very simple models that assume independent feature (for example as very simple model that determines loan approval based on income, debt, and credit scores)

2. Logistic Regression is very simple and easy to understand. It is very efficient and takes up little to no memory once the model is fully trained (keeps only the parameter weights to make a prediction).

3. The model assumes linear independent features which might not hold. In addition it cannot capture more complex interaction or higher level functionality on its own (would need to use a model such as neural networks to accomplish this).

4. Decided to use Logistic Regression since the model is easy to understand and very efficient.

### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [8]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))

### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [10]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

# TODO: Initialize the three models
clf_A = GaussianNB()
clf_B = DecisionTreeClassifier(random_state=0)
clf_C = AdaBoostClassifier(random_state=0)
clf_D = KNeighborsClassifier()
clf_E = SGDClassifier(random_state=0)
clf_F = SVC(random_state=0)
clf_G = LogisticRegression(random_state=0)

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]

# TODO: Execute the 'train_predict' function for each classifier and each training set size

# Gaussian Naive Bayes
print('\nGaussianNB: ')
train_predict(clf_A, X_train_100, y_train_100 , X_test, y_test)
train_predict(clf_A, X_train_200 , y_train_200 , X_test, y_test)
train_predict(clf_A, X_train, y_train, X_test, y_test)

# Decision Tree Classifier
print('\nDecisionTreeClassifier: ')
train_predict(clf_B, X_train_100, y_train_100 , X_test, y_test)
train_predict(clf_B, X_train_200 , y_train_200 , X_test, y_test)
train_predict(clf_B, X_train, y_train, X_test, y_test)

# AdaBoost Classifier
print('\nAdaBoostClassifier: ')
train_predict(clf_C, X_train_100, y_train_100 , X_test, y_test)
train_predict(clf_C, X_train_200 , y_train_200 , X_test, y_test)
train_predict(clf_C, X_train, y_train, X_test, y_test)

# K-Neighbors Classifier
print('\nKNeighborsClassifier: ')
train_predict(clf_D, X_train_100, y_train_100 , X_test, y_test)
train_predict(clf_D, X_train_200 , y_train_200 , X_test, y_test)
train_predict(clf_D, X_train, y_train, X_test, y_test)

# SGD Classifier
print('\nSGDClassifier: ')
train_predict(clf_E, X_train_100, y_train_100 , X_test, y_test)
train_predict(clf_E, X_train_200 , y_train_200 , X_test, y_test)
train_predict(clf_E, X_train, y_train, X_test, y_test)

# Support Vector Classifier
print('\nSVC: ')
train_predict(clf_F, X_train_100, y_train_100 , X_test, y_test)
train_predict(clf_F, X_train_200 , y_train_200 , X_test, y_test)
train_predict(clf_F, X_train, y_train, X_test, y_test)

# Logistic Regression Classifier
print('\nLogisticRegression: ')
train_predict(clf_G, X_train_100, y_train_100 , X_test, y_test)
train_predict(clf_G, X_train_200 , y_train_200 , X_test, y_test)
train_predict(clf_G, X_train, y_train, X_test, y_test)


GaussianNB: 
Training a GaussianNB using a training set size of 100. . .
Trained model in 0.0012 seconds
Made predictions in 0.0005 seconds.
F1 score for training set: 0.8467.
Made predictions in 0.0005 seconds.
F1 score for test set: 0.8029.
Training a GaussianNB using a training set size of 200. . .
Trained model in 0.0010 seconds
Made predictions in 0.0004 seconds.
F1 score for training set: 0.8406.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7244.
Training a GaussianNB using a training set size of 300. . .
Trained model in 0.0008 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 0.8038.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.7634.

DecisionTreeClassifier: 
Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0009 seconds
Made predictions in 0.0002 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0002 seconds.
F1 score for test set: 0.6154.
Training a DecisionTreeClass

### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

** Classifer 1 - Gaussian Naive Bayes**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0017 seconds          | 0.0005 seconds         | 0.8467           | 0.8029          |
| 200               | 0.0007 seconds          | 0.0003 seconds         | 0.8406           | 0.7244          |
| 300               | 0.0011 seconds          | 0.0006 seconds         | 0.8038           | 0.7634          |

** Classifer 2 - Decision Tree Classifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0017 seconds          | 0.0002 seconds         | 1.0000           | 0.6102          |
| 200               | 0.0029 seconds          | 0.0001 seconds         | 1.0000           | 0.7040          |
| 300               | 0.0023 seconds          | 0.0002 seconds         | 1.0000           | 0.6102          |

** Classifer 3 - AdaBoost Classifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.1595 seconds          | 0.0077 seconds         | 0.9481           | 0.7669          |
| 200               | 0.2515 seconds          | 0.0059 seconds         | 0.8927           | 0.8281          |
| 300               | 0.2582 seconds          | 0.0051 seconds         | 0.8637           | 0.7820          |

** Classifer 4 - K-Neighbor Classifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0005 seconds          | 0.0014 seconds         | 0.8060           | 0.7246          |
| 200               | 0.0006 seconds          | 0.0027 seconds         | 0.8800           | 0.7692          |
| 300               | 0.0010 seconds          | 0.0025 seconds         | 0.8809           | 0.7801          |

** Classifer 5 - Stochastic Gradient Descent Classifier**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0007 seconds          | 0.0003 seconds         | 0.7786           | 0.7328          |
| 200               | 0.0023 seconds          | 0.0002 seconds         | 0.4421           | 0.4884          |
| 300               | 0.0017 seconds          | 0.0008 seconds         | 0.0193           | 0.1250          |

** Classifer 6 - Support Vector Classifier **  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0013 seconds          | 0.0008 seconds         | 0.8777           | 0.7746          |
| 200               | 0.0035 seconds          | 0.0011 seconds         | 0.8679           | 0.7815          |
| 300               | 0.0102 seconds          | 0.0023 seconds         | 0.8761           | 0.7838          |

** Classifer 7 - Logistic Regression**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               | 0.0478 seconds          | 0.0003 seconds         | 0.8593           | 0.7647          |
| 200               | 0.0021 seconds          | 0.0002 seconds         | 0.8562           | 0.7914          |
| 300               | 0.0030 seconds          | 0.0004 seconds         | 0.8468           | 0.8060          |



## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

If we take the weight average of Guassian Naive Bayes, Decision Tree Classifier, and the Support Vector Classifier for training time we get 0.0011, 0.0024, and 0.0065 seconds respectively with GNB performing the best and SVC the worst. When looking at the weight time for prediction time we get 0.00048, 0.00017, and 0.00165 seconds with DT perform the best and again SVC the worst. Taking the average values of the various training sizes for each model we get F1 scores of 0.7636, 0.6415, 0.7799 with DT performing the worst and SVC the best.

It is important to note that the dataset is extremely small and with a lot of features that might benefit from additional cleaning and multiple trials to provide more robust and accurate models. In addition there is no discussion of the cost benefit analysis or direct trade-offs of the computational resources vs the need to properly diagnoise student interventions.

Based on the data seen so far I decided to use the Support Vector Classifier. The reason I decided to go with the SVC is that it provided the best average graduation score. Even though the computation time was higher than the other models, I believe it produces more reliable results for the small dataset which is a small price to pay in exchange giving student early assistance. Without proper preventions it might cost the school more money in the long run than the computational costs.

If however the dataset was much larger where the cost of computer resources was paramount than I would argue it might favor the Naive Bayes model which still pretty reliable and scales well. The decision tree does not seem to produce a robust model to be of further consideration.

### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

The Support Vector Classifier takes look at similar points in the dataset and tries to look for a boundary line that would maximize the distance between students that graduate from those who do not. In cases where the decision boundry line is complicated the SVC can reorganize the features to make a new boundary for distinct labels.

Once a decision boundary line is determined that separates the distinct qualities that determine graduation rates we can than use it to determine if new students might likely graduate or need more interventions based on which side of the boundary the new data point lines.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [11]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.model_selection import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.metrics import make_scorer, f1_score

# TODO: Create the parameters list you wish to tune
parameters = [{'C': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]}, {'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}, {'random_state': [0]}, {'random_state': [0]}]

# TODO: Initialize the classifier
clf = clf_F

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, greater_is_better = True, pos_label="yes")

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, param_grid = parameters, scoring = f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

# Report the final F1 score for training and testing after parameter tuning
print "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))

/Users/trdelgado/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


Made predictions in 0.0049 seconds.
Tuned model has a training F1 score of 0.8686.
Made predictions in 0.0017 seconds.
Tuned model has a testing F1 score of 0.7733.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*

**Answer: **

The final Support Vector Classifier has F1 scores for training and testing of 0.8686 & 0.7733 respectively with a predictive testing time of 0.0031 seconds. The tuned model does not seem to show any significant improvements over the default parameters in either the prediction time or F1 test score.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.